In [18]:
import random
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Latex
# display(Latex(f'$x_{i}$'))
def printL(code):
    display(Latex(f'${code}$'))

## function definitions

In [19]:
def poly_G(v):
    # Precompute common polynomial factor
    G = PF(1)
    for i in range(len(v)):
        G *= Y-v[i]
    return G

In [20]:
def poly_L(i,G,v):
    # Precompute Lagrangian polynomial
    L_i = G/(Y-v[i])
    denom = F(1)
    for h in range(len(v)):
        if h == i:
            continue
        denom *= v[i]-v[h]
    L_i /= denom
    return L_i

In [21]:
def sys_G_RS():
    # Compute a very systematic generator matrix
    eye = matrix.identity(F,k)
    G_RS = eye.augment(matrix.zero(F,nrows=k,ncols=n-k))
    G = poly_G(x[:k])
    #print(G)
    
    for i in range(k):
        L_i = poly_L(i,G,x[:k])
        G_RS[i,k:] = matrix(F,[L_i(x[j]) for j in range(k,n)])
    
    return G_RS

In [38]:
def RS_decoder(r):
    # r: received message
    # G_poly = poly_G(x)
    # L[i] = L_i = poly_L(i,G,x)
    # G_poly and L are precomputed globally
    
    R = r*L
    P = matrix(PF, [[G_poly,0],[-R,1]])
    
    while P[1,0].degree() >= (P[1,1].degree() + k - 1):
        (qP, rP) = P[0][0].quo_rem(P[1][0])
        P = matrix(PF, [ [P[1,0], P[1,1]], [rP, P[0,1] - qP*P[1,1]] ])
    
    if P[0,0].degree() < P[1,1].degree() + (k-1):
        f = -P[0,0]/P[0,1]
    else:
        f = -P[1,0]/P[1,1]
        
    return matrix(F,[f(x[i]) for i in range(n)])


In [46]:
def sim_chan(p,c):
    # p: cross-over probability
    # c: codeword sent through channel
    # r: received codeword
    # alpha: correct letter
    # beta: wrong letter
    # alphabet: global variable that contains F.list()
    
    r = vector(F,n)
    for i in range(n):
        if random.random() > p:
            r[i] = c[0,i]
        else:
            beta_idx = (alphabet.index(c[0,i]) + random.randint(1,q-1)) % q
            r[i] = alphabet[beta_idx]
    return r

In [29]:
def random_msg():
    return matrix([F.random_element() for i in range(k)])

## Simulations of decoding error rate

In [53]:
# probability simulation

q = 256; n = 255; k = 251
F.<X> = FiniteField(q)
PF.<Y> = PolynomialRing(F)
alphabet = F.list()
x = alphabet[1:]

G = sys_G_RS()

# Precomputing
G_poly = poly_G(x)
L = vector(PF,[poly_L(i,G_poly,x) for i in range(n)])

def simulate_errors(p,N):
    failures = 0
    for i in range(N):
        # T1 = time.time()
        m = random_msg()
        # print(f"random_msg: {time.time()-T1}")
        c = m*G

        # T1 = time.time()
        r = sim_chan(p,c)
        # print(f"sim_chan: {time.time()-T1}")

        # T1 = time.time()
        c_hat = RS_decoder(r)
        # print(f"RS_decoder: {time.time()-T1}")
        if c_hat != c:
            failures += 1

    print(f"Number of failures: {failures}")
    print(f"Failure rate: {failures/N}")
    return failures

print([simulate_errors(10^(-3),20) for i in range(3)])

Number of failures: 0
Failure rate: 0
Number of failures: 0
Failure rate: 0
Number of failures: 0
Failure rate: 0
[0, 0, 0]
